# Pre-Processing

In [3]:
%pip install pdfplumber
%pip install langchain
%pip install langchain_elasticsearch
%pip install langchain_huggingface
%pip install uuid
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os

#Get the file path of the pdf in the current directory
def get_pdf_path():
    list_of_files = os.listdir("./data_source/train_source")
    return list_of_files

In [18]:
import re
import pdfplumber

def pdf2text(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    pages = pdf.pages
    text = ""
    for page in pages:
        sub = page.extract_text()
        text += sub

    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)

    # .를 기준으로 문장단위 chunk 생성
    tokenized_sentence = cleaned_text.split(".")

    return tokenized_sentence

In [19]:
from uuid import uuid4

pdf_path = get_pdf_path()

parsed_text = []

for path in pdf_path:
    parsed_text.append(pdf2text("./data_source/train_source/"+path))

In [23]:
from langchain_core.documents import Document

documents = []

for single_pdf in parsed_text:
    for sen in single_pdf:
        doc = Document(
            page_content=sen,
            metadata={"source": f"{pdf_path}"},
        )
        documents.append(doc)

uuids = [str(uuid4()) for _ in range(len(documents))]

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore

In [26]:
embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-mpnet-base-v2",
    # model_kwargs={'device':'cuda'}
)

client = ElasticsearchStore(
    es_url="https://agi.veritasgarage.com:443", 
    index_name="dacon_nori_index", 
    es_user="elastic", 
    es_password="sukbeom", 
    embedding=embeddings)

client.add_documents(documents=documents, ids=uuids)